# Bermudan Swaption Pricing

This notebook replicates the QuantLib `BermudanSwaption` example.

**Models covered:**
- G2++ (two-factor Gaussian)
- Hull-White (one-factor mean-reverting)
- Black-Karasinski (lognormal short-rate)

In [1]:
import pyquantlib as ql
from datetime import date

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.QL_VERSION})")

PyQuantLib 0.1.0 (QuantLib 1.40)


## Market Data

Swaption volatility matrix for calibration (diagonal: 1x5, 2x4, 3x3, 4x2, 5x1).

In [2]:
swap_lengths = [1, 2, 3, 4, 5]
swaption_vols = [
    [0.1490, 0.1340, 0.1228, 0.1189, 0.1148],  # 1Y option
    [0.1290, 0.1201, 0.1146, 0.1108, 0.1040],  # 2Y option
    [0.1149, 0.1112, 0.1070, 0.1010, 0.0957],  # 3Y option
    [0.1047, 0.1021, 0.0980, 0.0951, 0.1270],  # 4Y option
    [0.1000, 0.0950, 0.0900, 0.1230, 0.1160],  # 5Y option
]

print("Calibration swaptions (diagonal):")
for i in range(5):
    j = 4 - i
    print(f"  {i+1}x{swap_lengths[j]}: {swaption_vols[i][j]:.2%}")

Calibration swaptions (diagonal):
  1x5: 11.48%
  2x4: 11.08%
  3x3: 10.70%
  4x2: 10.21%
  5x1: 10.00%


## Setup

In [3]:
# Using Python datetime.date - PyQuantLib converts automatically
today = date(2002, 2, 15)
settlement = date(2002, 2, 19)

# Set evaluation date (implicit conversion from datetime.date)
ql.Settings.evaluationDate = today

calendar = ql.TARGET()
day_counter = ql.Actual365Fixed()

# Flat yield term structure (hidden handle: pass quote directly)
flat_rate = ql.SimpleQuote(0.04875825)
term_structure = ql.FlatForward(settlement, flat_rate, day_counter)

print(f"Evaluation date: {today}")
print(f"Settlement date: {settlement}")
print(f"Flat rate:       {flat_rate.value():.4%}")

Evaluation date: 2002-02-15
Settlement date: 2002-02-19
Flat rate:       4.8758%


## Swap Conventions

In [4]:
# Shorter enum names
fixed_leg_frequency = ql.Annual
fixed_leg_convention = ql.Unadjusted
fixed_leg_day_counter = ql.Thirty360(ql.Thirty360.European)

floating_leg_frequency = ql.Semiannual
floating_leg_convention = ql.ModifiedFollowing

# Index (hidden handle: pass term structure directly)
index = ql.Euribor6M(term_structure)

print(f"Fixed leg:    {fixed_leg_frequency}, {fixed_leg_convention}")
print(f"Floating leg: {floating_leg_frequency}, {floating_leg_convention}")
print(f"Index:        {index.name()}")

Fixed leg:    Frequency.Annual, BusinessDayConvention.Unadjusted
Floating leg: Frequency.Semiannual, BusinessDayConvention.ModifiedFollowing
Index:        Euribor6M Actual/360


## Underlying Swap Definition

In [5]:
# Using Period strings: "1Y" instead of Period(1, Years)
start_date = calendar.advance(settlement, ql.Period("1Y"), floating_leg_convention)
maturity = calendar.advance(start_date, ql.Period("5Y"), floating_leg_convention)

# Schedules
fixed_schedule = ql.Schedule(
    start_date, maturity,
    ql.Period(fixed_leg_frequency),
    calendar,
    fixed_leg_convention, fixed_leg_convention,
    ql.DateGeneration.Forward, False
)

float_schedule = ql.Schedule(
    start_date, maturity,
    ql.Period(floating_leg_frequency),
    calendar,
    floating_leg_convention, floating_leg_convention,
    ql.DateGeneration.Forward, False
)

# Alternatively, use MakeSchedule with keyword arguments:
# fixed_schedule = ql.MakeSchedule(
#     effectiveDate=start_date,
#     terminationDate=maturity,
#     frequency=fixed_leg_frequency,
#     calendar=calendar,
#     convention=fixed_leg_convention,
# )

print(f"Swap start:    {start_date}")
print(f"Swap maturity: {maturity}")

Swap start:    February 19th, 2003
Swap maturity: February 19th, 2008


In [6]:
# Dummy swap to find fair rate
notional = 1000.0

dummy_swap = ql.VanillaSwap(
    ql.SwapType.Payer, notional,
    fixed_schedule, 0.03, fixed_leg_day_counter,
    float_schedule, index, 0.0, index.dayCounter()
)
dummy_swap.setPricingEngine(ql.DiscountingSwapEngine(term_structure))

atm_rate = dummy_swap.fairRate()
otm_rate = atm_rate * 1.2
itm_rate = atm_rate * 0.8

print(f"ATM rate: {atm_rate:.4%}")
print(f"OTM rate: {otm_rate:.4%} (20% above ATM)")
print(f"ITM rate: {itm_rate:.4%} (20% below ATM)")

ATM rate: 5.0000%
OTM rate: 6.0000% (20% above ATM)
ITM rate: 4.0000% (20% below ATM)


In [7]:
# Create ATM, OTM, ITM swaps
atm_swap = ql.VanillaSwap(
    ql.SwapType.Payer, notional,
    fixed_schedule, atm_rate, fixed_leg_day_counter,
    float_schedule, index, 0.0, index.dayCounter()
)

otm_swap = ql.VanillaSwap(
    ql.SwapType.Payer, notional,
    fixed_schedule, otm_rate, fixed_leg_day_counter,
    float_schedule, index, 0.0, index.dayCounter()
)

itm_swap = ql.VanillaSwap(
    ql.SwapType.Payer, notional,
    fixed_schedule, itm_rate, fixed_leg_day_counter,
    float_schedule, index, 0.0, index.dayCounter()
)

## Calibration Helpers

In [9]:
# Period strings for maturities
swaption_maturities = [ql.Period(f"{i}Y") for i in range(1, 6)]

def date_to_time(d):
    return day_counter.yearFraction(settlement, d)

# Create calibration helpers and collect mandatory times
swaptions = []
mandatory_times = set()

for i in range(5):
    j = 4 - i  # Diagonal: 1x5, 2x4, 3x3, 4x2, 5x1
    vol = ql.SimpleQuote(swaption_vols[i][j])
    
    # Period string for swap length
    helper = ql.SwaptionHelper(
        swaption_maturities[i],
        ql.Period(f"{swap_lengths[j]}Y"),
        vol,
        index,
        index.tenor(),
        index.dayCounter(),
        index.dayCounter(),
        term_structure,  # Hidden handle
    )
    swaptions.append(helper)
    
    # Collect times from underlying instruments
    swp = helper.swaption()
    underlying = helper.underlying()
    
    for d in swp.exercise().dates():
        mandatory_times.add(date_to_time(d))
    for cf in underlying.fixedLeg():
        mandatory_times.add(date_to_time(cf.date()))
    for cf in underlying.floatingLeg():
        mandatory_times.add(date_to_time(cf.date()))

print(f"Created {len(swaptions)} calibration helpers")
print(f"Collected {len(mandatory_times)} mandatory time points")

Created 5 calibration helpers
Collected 25 mandatory time points


## Time Grid

In [10]:
grid = ql.TimeGrid(sorted(mandatory_times), 30)

print(f"Time grid: {len(grid)} points, range [{grid.front():.2f}, {grid.back():.2f}]")

Time grid: 40 points, range [0.00, 6.02]


## Model Definitions

Hidden handles: pass term structure directly.

In [12]:
# Models with hidden handles
model_g2 = ql.G2(term_structure)
model_hw_analytic = ql.HullWhite(term_structure)
model_hw_numerical = ql.HullWhite(term_structure)
model_bk = ql.BlackKarasinski(term_structure)

print("Models created:")
print("  - G2++")
print("  - Hull-White x2 (analytic + numerical)")
print("  - Black-Karasinski")

Models created:
  - G2++
  - Hull-White x2 (analytic + numerical)
  - Black-Karasinski


## Calibration

In [13]:
def calibrate_model(model, helpers, engine_fn, name):
    """Calibrate model and show results."""
    print(f"\n{name}")
    print("-" * 50)
    
    for h in helpers:
        h.setPricingEngine(engine_fn(model))
    
    optimizer = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
    end_criteria = ql.EndCriteria(400, 100, 1e-8, 1e-8, 1e-8)
    model.calibrate(helpers, optimizer, end_criteria)
    
    # Show calibration quality
    print(f"{'Swaption':<10} {'Model':>10} {'Market':>10} {'Error':>10}")
    for i, helper in enumerate(helpers):
        j = 4 - i
        model_value = helper.modelValue()
        implied_vol = helper.impliedVolatility(model_value, 1e-4, 1000, 0.05, 0.50)
        market_vol = swaption_vols[i][j]
        error = implied_vol - market_vol
        print(f"{i+1}x{swap_lengths[j]:<8} {implied_vol:>10.4%} {market_vol:>10.4%} {error:>+10.4%}")
    
    return model.params()

In [14]:
# G2++ (analytic)
params = calibrate_model(
    model_g2, swaptions,
    lambda m: ql.G2SwaptionEngine(m, 6.0, 16),
    "G2++ (analytic)"
)
print(f"\nParams: a={params[0]:.4f}, sigma={params[1]:.4f}, b={params[2]:.4f}, eta={params[3]:.4f}, rho={params[4]:.4f}")


G2++ (analytic)
--------------------------------------------------
Swaption        Model     Market      Error
1x5          10.0455%   11.4800%   -1.4345%
2x4          10.5123%   11.0800%   -0.5677%
3x3          10.7050%   10.7000%   +0.0050%
4x2          10.8382%   10.2100%   +0.6282%
5x1          10.9439%   10.0000%   +0.9439%

Params: a=0.0501, sigma=0.0095, b=0.0501, eta=0.0095, rho=-0.7636


In [15]:
# Hull-White (Jamshidian - analytic)
params = calibrate_model(
    model_hw_analytic, swaptions,
    lambda m: ql.JamshidianSwaptionEngine(m),
    "Hull-White (Jamshidian)"
)
print(f"\nParams: a={params[0]:.4f}, sigma={params[1]:.4f}")


Hull-White (Jamshidian)
--------------------------------------------------
Swaption        Model     Market      Error
1x5          10.6204%   11.4800%   -0.8596%
2x4          10.6296%   11.0800%   -0.4504%
3x3          10.6341%   10.7000%   -0.0659%
4x2          10.6443%   10.2100%   +0.4343%
5x1          10.6613%   10.0000%   +0.6613%

Params: a=0.0464, sigma=0.0059


In [16]:
# Hull-White (Tree - numerical)
params = calibrate_model(
    model_hw_numerical, swaptions,
    lambda m: ql.TreeSwaptionEngine(m, grid),
    "Hull-White (Tree)"
)
print(f"\nParams: a={params[0]:.4f}, sigma={params[1]:.4f}")


Hull-White (Tree)
--------------------------------------------------
Swaption        Model     Market      Error
1x5          10.2928%   11.4800%   -1.1872%
2x4          10.5454%   11.0800%   -0.5346%
3x3          10.6562%   10.7000%   -0.0438%
4x2          10.7368%   10.2100%   +0.5268%
5x1          10.8226%   10.0000%   +0.8226%

Params: a=0.0560, sigma=0.0061


In [17]:
# Black-Karasinski (Tree - numerical)
params = calibrate_model(
    model_bk, swaptions,
    lambda m: ql.TreeSwaptionEngine(m, grid),
    "Black-Karasinski (Tree)"
)
print(f"\nParams: a={params[0]:.4f}, sigma={params[1]:.4f}")


Black-Karasinski (Tree)
--------------------------------------------------
Swaption        Model     Market      Error
1x5          10.3067%   11.4800%   -1.1733%
2x4          10.5643%   11.0800%   -0.5157%
3x3          10.6661%   10.7000%   -0.0339%
4x2          10.7338%   10.2100%   +0.5238%
5x1          10.8033%   10.0000%   +0.8033%

Params: a=0.0443, sigma=0.1207


## Bermudan Swaption Setup

In [18]:
# Exercise dates from fixed leg accrual start dates
bermudan_dates = [cf.accrualStartDate() for cf in atm_swap.fixedLeg()]
bermudan_exercise = ql.BermudanExercise(bermudan_dates)

# Convert to Python dates for display
print(f"Exercise dates ({len(bermudan_dates)}):")
for d in bermudan_dates:
    print(f"  {d.to_date()}")  # .to_date() converts to datetime.date

Exercise dates (5):
  2003-02-19
  2004-02-19
  2005-02-19
  2006-02-19
  2007-02-19


In [19]:
atm_bermudan = ql.Swaption(atm_swap, bermudan_exercise)
otm_bermudan = ql.Swaption(otm_swap, bermudan_exercise)
itm_bermudan = ql.Swaption(itm_swap, bermudan_exercise)

## Bermudan Swaption Pricing

In [20]:
def price_bermudan(swaption, strike_type, rate):
    """Price with all engines."""
    print(f"\n{strike_type} (strike {rate:.4%})")
    print("-" * 50)
    
    engines = [
        ("G2 (tree)", ql.TreeSwaptionEngine(model_g2, 50)),
        ("G2 (fdm)", ql.FdG2SwaptionEngine(model_g2)),
        ("HW (tree)", ql.TreeSwaptionEngine(model_hw_analytic, 50)),
        ("HW (fdm)", ql.FdHullWhiteSwaptionEngine(model_hw_analytic)),
        ("HW num (tree)", ql.TreeSwaptionEngine(model_hw_numerical, 50)),
        ("HW num (fdm)", ql.FdHullWhiteSwaptionEngine(model_hw_numerical)),
        ("BK (tree)", ql.TreeSwaptionEngine(model_bk, 50)),
    ]
    
    results = []
    for name, engine in engines:
        swaption.setPricingEngine(engine)
        npv = swaption.NPV()
        results.append((name, npv))
        print(f"{name:<20} {npv:>10.4f}")
    
    return results

In [21]:
atm_results = price_bermudan(atm_bermudan, "ATM", atm_rate)


ATM (strike 5.0000%)
--------------------------------------------------
G2 (tree)               14.1319
G2 (fdm)                14.1127
HW (tree)               12.9284
HW (fdm)                12.9095
HW num (tree)           13.1452
HW num (fdm)            13.1192
BK (tree)               13.0168


In [22]:
otm_results = price_bermudan(otm_bermudan, "OTM", otm_rate)


OTM (strike 6.0000%)
--------------------------------------------------
G2 (tree)                3.3024
G2 (fdm)                 3.1808
HW (tree)                2.5139
HW (fdm)                 2.4596
HW num (tree)            2.6157
HW num (fdm)             2.5608
BK (tree)                3.2732


In [23]:
itm_results = price_bermudan(itm_bermudan, "ITM", itm_rate)


ITM (strike 4.0000%)
--------------------------------------------------
G2 (tree)               42.6040
G2 (fdm)                42.7055
HW (tree)               42.2515
HW (fdm)                42.2153
HW num (tree)           42.3464
HW num (fdm)            42.2983
BK (tree)               41.8117


## Summary

In [24]:
print("\n" + "=" * 60)
print("SUMMARY: Bermudan Swaption Prices")
print("=" * 60)
print(f"\n{'Engine':<20} {'ATM':>12} {'OTM':>12} {'ITM':>12}")
print("-" * 56)

for i, (engine, _) in enumerate(atm_results):
    print(f"{engine:<20} {atm_results[i][1]:>12.4f} {otm_results[i][1]:>12.4f} {itm_results[i][1]:>12.4f}")


SUMMARY: Bermudan Swaption Prices

Engine                        ATM          OTM          ITM
--------------------------------------------------------
G2 (tree)                 14.1319       3.3024      42.6040
G2 (fdm)                  14.1127       3.1808      42.7055
HW (tree)                 12.9284       2.5139      42.2515
HW (fdm)                  12.9095       2.4596      42.2153
HW num (tree)             13.1452       2.6157      42.3464
HW num (fdm)              13.1192       2.5608      42.2983
BK (tree)                 13.0168       3.2732      41.8117
